<a href="https://colab.research.google.com/github/5655530/AIFFEL_CORE_QUEST/blob/main/CNN_%EA%B5%AC%EC%A1%B0_%EA%B5%AC%ED%98%84%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN 구조 구현하기**
---
tf.keras 라이브러리를 이용해서 CNN 구조를 구현해봅시다.
2D convolutional layer는 tf.keras.layers.Conv2D 객체를 생성하여 구현할 수 있습니다.

Conv2D 객체의 자료형은 keras.engine.keras_tensor.KerasTensor입니다.
모델을 생성할 때, tf.keras.layers.Input 레이어를 맨 처음에 정의해야 합니다. 여기서는 (28, 28, 1) 크기의 데이터를 입력받는 input layer를 생성했습니다.

input_layer.shape 결과에서 처음 보이는 None은 데이터셋의 batch size에 해당됩니다. x라는 레이어 객체는 output 채널이 4개인 convolutional layer이므로 x.shape의 결과는 (None, 28, 28, 4)가 됩니다.

- 중요한 점은, 레이어를 생성할 때 output = Conv2D(params)(input) 형태로 input과 output 변수를 넣고, 이전 레이어의 output을 다음 레이어의 input으로 설정함으로써 레이어들이 서로 연결되도록 해야 한다는 것입니다.

In [1]:
import tensorflow as tf

In [2]:
# 가로 28, 세로 28, 채널 수 1의 input 데이터를 받는 input layer 생성하기
input_layer = tf.keras.layers.Input(shape=(28, 28, 1))

# 커널의 가로 세로 사이즈는 3이고, 채널 수는 4, zero-padding을 넣고,
# stride는 1로 한 Conv2D layer
x = tf.keras.layers.Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation='relu')(input_layer)

print(type(x))
print(x)

<class 'keras.engine.keras_tensor.KerasTensor'>
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 4), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")


In [3]:
print(input_layer.shape)
print(x.shape)

(None, 28, 28, 1)
(None, 28, 28, 4)


## **pooling layer가 포함된 CNN 모델 구현**
---
이번에는 pooling layer가 포함된 CNN 모델을 만들어봅시다.
(27, 27, 1) 크기의 입력을 받는 input_tensor와 Conv2D 레이어 x1, 그리고 pooling layer에 해당하는 MaxPooling2D 레이어 x2로 구성되어 있습니다.

In [4]:
# 가로 27, 세로 27, 채널 수 1의 input 데이터를 받는 input layer 생성하기
input_tensor = tf.keras.layers.Input(shape=(27, 27, 1))

In [7]:
# 커널의 가로 세로 사이즈는 2이고, 채널 수는 6, zero-padding을 넣고,
# stride는 2인 Conv2D layer (활성화 함수 : relu)
x1 = tf.keras.layers.Conv2D(filters=6, kernel_size=2, strides=2, padding='same', activation='relu')(input_tensor)

In [8]:
# 가로 세로 사이즈가 2인 영역에서 최대값을 뽑는 Maxpooling을 적용
x2 = tf.keras.layers.MaxPooling2D(2)(x1)
print(x1)
print(x2)

KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 6), dtype=tf.float32, name=None), name='conv2d_2/Relu:0', description="created by layer 'conv2d_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 6), dtype=tf.float32, name=None), name='max_pooling2d_1/MaxPool:0', description="created by layer 'max_pooling2d_1'")


레이어 객체를 쭉 정의한 다음, tf.keras.Model 함수에 input과 output을 연결해서 모델 객체를 생성합니다. model.summary() 함수를 이용하면 완성된 모델의 구조를 한번에 살펴볼 수 있습니다.

In [9]:
input_layer = tf.keras.layers.Input(shape=(7, 7, 5))
convlayer1 = tf.keras.layers.Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(input_layer)
convlayer2 = tf.keras.layers.Conv2D(filters=8, kernel_size=3, strides=1, padding='valid')(convlayer1)
pooling = tf.keras.layers.MaxPooling2D(2)(convlayer2)

model = tf.keras.Model(inputs=input_layer, outputs=pooling)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 7, 7, 5)]         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 4)           184       
                                                                 
 conv2d_4 (Conv2D)           (None, 5, 5, 8)           296       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 8)          0         
 2D)                                                             
                                                                 
Total params: 480
Trainable params: 480
Non-trainable params: 0
_________________________________________________________________


In [11]:
input_layer = tf.keras.layers.Input(shape=(13, 13, 5))
convlayer1 = tf.keras.layers.Conv2D(filters=8, kernel_size=5, strides=2, padding='valid')(input_layer)
convlayer2 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=2, padding='valid')(convlayer1)
pooling = tf.keras.layers.MaxPooling2D(2)(convlayer2)

model = tf.keras.Model(inputs=input_layer, outputs=pooling)
model.summary()

ValueError: ignored

이 코드가 오류가 나는 이유는 두 번째 Conv2D 레이어의 output convlayer2의 크기가 작아서 pooling 연산을 수행할 공간이 부족하기 때문입니다.

첫 번째 Conv2D 레이어의 output convlayer1의 크기는 5x5이고, 두 번째 Conv2D 레이어의 output convlayer2의 크기는 1x1입니다. 1x1를 pooling 연산을 하기에는 공간이 부족합니다.

## **이미지 분류 CNN 모델**
---
CNN 모델을 이미지 분류에 사용하려면, Conv2D 레이어의 output인 3차원 feature map에 flatten을 적용한 다음, fully connected layer(Dense)를 연결해야 합니다.

In [12]:
input_tensor = tf.keras.layers.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2)(x)

# 3차원으로 되어있는 Feature map 결과를 Fully Connected 연결하기 위해서는 Flatten()을 적용해야 합니다.
x = tf.keras.layers.Flatten()(x)

# Flatten 된 결과를 100의 노드를 가진 Fuly Connected Layer와 연결
x = tf.keras.layers.Dense(100, activation='relu')(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)

In [14]:
# tf.keras.Model을 사용하여 model을 정의
model = tf.keras.Model(inputs=input_tensor, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_10 (Conv2D)          (None, 26, 26, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10816)             0         
                                                                 
 dense (Dense)               (None, 100)               1081700   
                                                           